In [1]:

from __future__ import absolute_import, division, unicode_literals


import model_V3 as Model

import sys
import logging
import numpy as np
import time
import os
import tensorflow as tf
#tf.enable_eager_execution()


In [2]:
"""
Clone GenSen repo here: https://github.com/Maluuba/gensen.git
And follow instructions for loading the model used in batcher
"""




"""

Loading Encoding Document
"""

print('build data')
COMMON_VOCAB_FILE = 'words.txt'
GERMAN_VOCAB_FILE = 'german_words.txt'
TREE_VOCAB_FILE = 'tree_words.txt'


def get_model():
    vocab = tf.contrib.lookup.index_table_from_file(COMMON_VOCAB_FILE, num_oov_buckets=1)
    vocab_german = tf.contrib.lookup.index_table_from_file(GERMAN_VOCAB_FILE,num_oov_buckets=1)
    vocab_tree  = tf.contrib.lookup.index_table_from_file(TREE_VOCAB_FILE,num_oov_buckets=1)
    
    model = Model.Model(30002,30002,76)   
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.002)
    checkpoint_directory = '../../models_checkpoints/common/'
    chkpnt = tf.contrib.eager.Checkpoint(optimizer=optimizer, model=model, optimizer_step=tf.train.get_or_create_global_step())
    print(chkpnt.restore(tf.train.latest_checkpoint(checkpoint_directory)))
    #log_msg("Restored Model upto Previous Checkpoint")
    
    vocabs = {
        "English" : vocab,
        "German " : vocab_german,
        "Tree ":vocab_tree
    }
    return model, vocabs


build data


In [3]:
model,vocabs = get_model()
np.save('../data/Emb.npy',model.rnn.W.numpy())

In [4]:
# # Set PATHs

PATH_TO_SENTEVAL = '../'
PATH_TO_DATA = '../data'

# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)

import senteval

# SentEval prepare and batcher
def prepare(params, samples):
    return samples

def encoder(*args):
    pass

def batcher(params, batch):
    sentences = []
    def process():
        for sample in batch:
            sentence = ' '.join(sample)
            sentence = tf.convert_to_tensor(sentence,dtype=tf.string)
            string = tf.string_split([sentence]).values
            str_idx = vocabs["English"].lookup(string)
            len_string = tf.cast(tf.size(str_idx),dtype=tf.int64)
            yield str_idx,len_string
    ds = tf.data.Dataset.from_generator(
      process, (tf.int64, tf.int64), (tf.TensorShape([None]), tf.TensorShape([])))
    ds = ds.padded_batch(batch_size=256, padded_shapes=([None],[]),padding_values=
                         (tf.convert_to_tensor(0,dtype=tf.int64),tf.convert_to_tensor(0,dtype=tf.int64)))
    ds = iter(ds)
    emb = []
    for d in ds:
        fs = model.rnn.frwrd(d[0],d[1],False)
        emb.append(fs.numpy())
    return np.vstack(emb)

# Set params for SentEval
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False}
params_senteval['classifier'] = {}

params_senteval['gensen'] = encoder

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks = [
                      # 'STS12', 'STS13', 'STS14', 'STS15', 'STS16',
                      # 'TREC', 
                      # 'STSBenchmark'
                      'MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'SST5', 
                      'MRPC',
                      'SICKEntailment', 
                      'SICKRelatedness'
                      ]
                      # 'Length', 'WordContent', 'Depth', 'TopConstituents',
                      # 'BigramShift', 'Tense', 'SubjNumber', 'ObjNumber',
                      # 'OddManOut', 'CoordinationInversion']
    results = se.eval(transfer_tasks)
    print('Completed Generating Data')


2018-10-12 05:59:12,149 : ***** Transfer task : MR *****


2018-10-12 05:59:12,286 : Generating sentence embeddings
2018-10-12 05:59:35,840 : Generated sentence embeddings
2018-10-12 05:59:36,073 : ***** Transfer task : CR *****


2018-10-12 05:59:36,091 : Generating sentence embeddings
2018-10-12 05:59:44,056 : Generated sentence embeddings
2018-10-12 05:59:44,077 : ***** Transfer task : MPQA *****


2018-10-12 05:59:44,100 : Generating sentence embeddings
2018-10-12 06:00:01,906 : Generated sentence embeddings
2018-10-12 06:00:02,055 : ***** Transfer task : SUBJ *****


2018-10-12 06:00:02,264 : Generating sentence embeddings
2018-10-12 06:00:25,956 : Generated sentence embeddings
2018-10-12 06:00:26,185 : ***** Transfer task : SST Binary classification *****


2018-10-12 06:00:26,740 : Computing embedding for train
2018-10-12 06:02:46,366 : Computed train embeddings
2018-10-12 06:02:46,381 : Computing embedding for dev
2018-10-12 06:02:48,840 : Computed dev embeddings
2018-10-12 06:

Completed Generating Data
